# Obtener Embeddings

In [ ]:
# Importación de las bibliotecas necesarias
import os
import pandas as pd
import numpy as np
import json
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import pickle
import dask.dataframe as dd
import multiprocessing

In [ ]:
# Configuración de OpenAI
openai.api_key = os.getenv('OPENAI_KEY') # reemplaza por tu API Key

## Sección de carga y preprocesamiento de datos

In [ ]:
# Ruta del archivo de datos
path_data = 'meta_All_Beauty.json'

In [ ]:
# Función para cargar los datos del archivo JSON
def load_data(path):
    data = []
    with open(path, 'r') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return data

In [ ]:
# Función para procesar la lista de datos en formato JSON
def process_json(json_data):
    for item in json_data:
        description = ' '.join(item['description'])
        item['description'] = description
        category = ','.join(item['category'])
        item['category'] = category    
    return json_data

In [ ]:
# Carga de datos
data_json = load_data(path_data)

# Procesamiento de datos
data_json = process_json(data_json)[:500] # Aquí solo se toman los primeros 500 registros si quieres puedes quitar esta restricción y ejecutarlo para los 32k productos

# Creación de un DataFrame
df = pd.DataFrame(data_json)

## Sección de generación de embeddings

In [ ]:
# Función para crear el embedding de un producto
def get_embedding_product(title, description):
    max_len = 8190 # Esto debido a que OpenAI tiene un limite de 8191
    data = f'title: {title} description:{description}'[:max_len]
    return get_embedding(data, engine='text-embedding-ada-002')

In [ ]:
# Creación de un Dask DataFrame para el cálculo paralelo
n_cores = multiprocessing.cpu_count()
ddf = dd.from_pandas(df, npartitions=n_cores)

In [ ]:
%%time
ddf['embedding'] = ddf.map_partitions(lambda df: df.apply(lambda row: get_embedding_product(row['title'], row['description']), axis=1)).compute(scheduler='threads')
# Esta celda para los 32k tuvo las siguientes metricas
# CPU times: total: 8.67 s
# Wall time: 18min 42s

In [ ]:
df = ddf.compute()

## Sección de almacenamiento

In [ ]:
# Lo guardamos para no tener que volver a calcular los embeddings y reutilizarlos
path_save = 'meta_All_Beauty.pickle'
df.to_pickle(path_save) 

# Playground

In [ ]:
# Importación de las bibliotecas necesarias
import os
import pandas as pd
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import pickle
from pandas import option_context

In [ ]:
# Configuración de OpenAI
openai.api_key = os.getenv('OPENAI_KEY') # reemplaza por tu API Key

In [ ]:
# Leyendo el DataFrame del archivo
path_save = 'processed.pickle'
df = pd.read_pickle(path_save)

In [ ]:
# Función para realizar una búsqueda
def search(query, data, n_results=15):
    search_embed = get_embedding(query, engine="text-embedding-ada-002")
    data["similarity"] = data['embedding'].apply(lambda x: cosine_similarity(x, search_embed))
    data = data.sort_values("similarity", ascending=False)
    return data.iloc[:n_results]

In [ ]:
%%time
results = search("crema facial", df)

In [ ]:
pd.set_option('display.max_colwidth', None)
results[['title', 'description']]

In [ ]:
pd.reset_option('display.max_colwidth')